In [65]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import datetime
from urllib.request import urlopen, Request

In [67]:
r = Request('https://th.investing.com/economic-calendar/', headers={'User-Agent': 'Mozilla/5.0'})
time.sleep(2)
response = urlopen(r).read()
soup = BeautifulSoup(response, "html.parser")
table = soup.find_all(class_ = "js-event-item")
result = []

In [ ]:
def event_calendar():
    
    for bl in table:
        event_datetime = bl.get('data-event-datetime', '').strip()
        currency = bl.find(class_="left flagCur noWrap").text.strip()
        intensity_divs = bl.find_all(class_="left textNum sentiment noWrap")
        event = bl.find(class_="left event").text.strip()
        intencity_val = 0
        true_count = 0

        for intence in intensity_divs:
            _true = intence.find_all(class_="grayFullBullishIcon")
            _false = intence.find_all(class_="grayEmptyBullishIcon")

            true_count = len(_true)

            if true_count == 3:
                intencity_val = 3
            elif true_count == 2:
                intencity_val = 2
            else :
                intencity_val = 1
                
        event_datetime = event_datetime.split(' ')
        date=event_datetime[0]
        time=event_datetime[1]
        
        
        result.append({'currency' : currency, 'date' : date, 'time' : time, 'intensity' : intencity_val, 'event':event})

    return result

In [75]:
news = event_calendar()
news_df = pd.DataFrame(news)

In [76]:
display(news_df.shape)
display(news_df.head(10))

(66, 5)

,currency,date,time,intensity,event
0,USD,2025/05/20,00:15:00,1,กำหนดแถลงของ โลแกน
1,USD,2025/05/20,00:30:00,2,สุนทรพจน์ของนายล็อกฮาร์ต (Kashkari) กรรมการของ...
2,CNY,2025/05/20,08:00:00,2,อัตราดอกเบี้ยเงินกู้ลูกค้าชั้นดีของธนาคาร PBoC...
3,CNY,2025/05/20,08:15:00,2,อัตราดอกเบี้ยเงินกู้ลูกค้าชั้นดีของธนาคาร PBoC
4,AUD,2025/05/20,11:30:00,3,การตัดสินใจเกี่ยวกับอัตราดอกเบี้ย ( พ.ค.)
5,AUD,2025/05/20,11:30:00,2,การแถลงการณ์นโยบายการเงินจากธนาคารกลางแห่งออสเ...
6,AUD,2025/05/20,11:30:00,2,การแถลงการณ์อัตราดอกเบี้ยธนาคารกลางออสเตรเลีย
7,EUR,2025/05/20,13:00:00,2,ดัชนีผู้ผลิต (PPI) เยอรมนี (เดือนต่อเดือน) (เ...
8,EUR,2025/05/20,13:00:00,1,ดัชนีราคาผู้ผลิต (PPI) ของเยอรมนี (ปีต่อปี) (...
9,GBP,2025/05/20,15:00:00,2,คำกล่าวของพิล (Pill) สมาชิก MPC ของ BoE


In [77]:
now = datetime.datetime.now()
date_time = now.strftime("%Y-%m-%d %H-%M-%S").strip().replace(' ', '_')
news_df.to_csv(f'Data/Event_Calendar/{date_time}.csv', index=False)